In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [32]:
import string
import re
import numpy as np
from nltk.corpus import words
from nltk.tokenize import word_tokenize
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [34]:
def simplify(word):
	dump = ''
	temp = []
	listOfWords = list(filter(None,re.split("([A-Z][^A-Z]*)",word)))
	if len(listOfWords) == len(word):
		return word.lower()
	for i in range(len(listOfWords)):
		listOfWords[i] = listOfWords[i].lower()
		if len(listOfWords[i]) == 1:
			dump = dump + listOfWords[i]
			if dump in words.words() and len(dump) > 2:
				temp.append(dump)
				dump = ''
		else:
			temp.append(listOfWords[i])
	return temp

def createTokens(data):
		listOfTweets = []
		listOfStances = []
		tweetVector = []
		for ind, row in data.iterrows():
			
			# Create a sentence using target and the tweet. Word vector will be formed from this.
			example_sentence = str(row["Target"]) + " " + str(row["Tweet"])

			# Remove punctuation
			final_sentence = example_sentence.translate(string.punctuation)

			wordList = word_tokenize(final_sentence)
			finalList = []
			s = ' '.join([i for i in wordList if i.isalpha()])

			# create tokens from the string and stem them
			wordList = word_tokenize(s)

			

			for word in wordList:
				#to break any combined word into its components for eg, hashtags
				finalList += simplify(word)

			final_sentence = ' '.join(finalList)
			listOfTweets.append(final_sentence)
			listOfStances.append(row["Stance"])

		return listOfTweets
 


def labelStance(labelDict, data):
	for key, val in labelDict.items():
		data.loc[data["Stance"] == val, "Stance"] = int(key)
	return data

trainTweets = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/stance-detection/train.csv').dropna(axis=1)
testTweets = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/stance-detection/test.csv').dropna(axis=1)

labelDict = {0:"AGAINST", 1:"FAVOR", 2:"NONE"}
trainTweets = labelStance(labelDict, trainTweets)
testTweets = labelStance(labelDict, testTweets)   
# print(trainTweets.shape)
# print(testTweets.shape)

listOfTweets = pd.concat([trainTweets, testTweets], axis=0)
# print(listOfTweets.shape)

listOfTweets = createTokens(listOfTweets)
print(listOfTweets.shape)

KeyboardInterrupt: ignored